# Lab 4 : CS 6760

## Introduction to Privacy Engineering, Summer 2021


## Fairness and Privacy

##### Installation:

Same as Lab 1. <br>No additional packages are required.

##### This lab contains 12 questions.
The first 11 questions are coding questions. <br>
The last question can be answered in words. 

##### Answer format

You can use the coding cells for your computations.\
Enter the answer obtained in the final markdown cell in this notebook.\
Numeric values can be rounded off upto 2 decimal places.

##### Grading rubric: for each question,
100% of the points - Correct code, correct output<br>
50% of the points - Minor logical error, partial output<br>
0% of the points - No attempt, incomplete code, wrong output

##### Submission instructions

You will have to submit the completed jupyter notebook file (.ipynb) in Canvas. <br> 
Please rename the submission file in the 'FirstName-LastName-Lab3.ipynb'format.

##### Acknowledgement

Data source: https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset

***
***

Importing libraries and functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from google.colab import drive

We will be using the 'Default of Credit Card Clients Dataset' for this Lab assignment. \
\
This dataset contains information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. \
\
Contents:
There are 25 variables:

- ID: ID of each client
- LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
- SEX: Gender (1=male, 2=female)
- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE: Marital status (1=married, 2=single, 3=others)
- AGE: Age in years
- PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
- PAY_2: Repayment status in August, 2005 (scale same as above)
- PAY_3: Repayment status in July, 2005 (scale same as above)
- PAY_4: Repayment status in June, 2005 (scale same as above)
- PAY_5: Repayment status in May, 2005 (scale same as above)
- PAY_6: Repayment status in April, 2005 (scale same as above)
- BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
- BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
- BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
- BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
- BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
- BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
- PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
- PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
- PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
- PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
- PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
- PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
- default.payment.next.month: Default payment (1=yes, 0=no)


In [2]:
drive.mount('/content/gdrive')
credit = pd.read_csv(r'/content/gdrive/MyDrive/DS5110/UCI_Credit_Card.csv')
credit

Mounted at /content/gdrive


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


We will be using this dataset to predict if a user will have a default payment in the next month or not. In the dataset, this information is available in the 'default.payment.next.month' feature.

In [3]:
credit.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

Let's answer some preliminary questions about the dataset.

##### Question 1:  ( 2 points )

How many data entry (rows) are present in this dataset

In [4]:
## Question 1
## YOUR CODE HERE
credit.shape[0]

30000

##### Question 2:  ( 2 points )

How many users are atleast 70 years old?

In [5]:
## Question 2
## YOUR CODE HERE
user_age = credit[credit['AGE'] >= 70]
user_age.shape[0]

25

##### Question 3: ( 2 points )

How many users are younger than 25 years?

In [6]:
## Question 3
## YOUR CODE HERE
user_age = credit[credit['AGE'] < 25]
user_age.shape[0]

2685

Default refers to the failure to repay a debt including interest or principal on a loan or security. A default can occur when a borrower is unable to make timely payments, misses payments, or avoids or stops making payments. In the dataset, this information is available in the 'default.payment.next.month' feature.

##### Question 4: ( 2 points )
What percentage of the users don't have a default payment?\
(1=yes, 0=no)

In [7]:
## Question 4
## YOUR CODE HERE
#Look up the sum number of no default payment and on default payment in "default.payment.next.month" column
non_default = credit['default.payment.next.month'].isin([0]).sum()
default_payment = credit['default.payment.next.month'].isin([1]).sum()
print("0 =", non_default)
print("1 = ", default_payment)
none_default = 100 - round(credit['default.payment.next.month'].mean() * 100, 2)
print("The percentage of the users don't have a default payment: {:0.2f}%".format(none_default))

0 = 23364
1 =  6636
The percentage of the users don't have a default payment: 77.88%


##### Question 5: ( 2 points )
Among all users atleast 70 years old,
What percentage of users don't have a default payment? 

In [8]:
## Question 5
## YOUR CODE HERE
# non_default = 100 - round(credit['default.payment.next.month'].mean() * 100, 2)
# user_age_70 = credit[credit['AGE'] >= 70]
# nre_df = user_age_70.groupby
# credit.groupby('AGE')['default.payment.next.month'].mean()
# test = credit.groupby('AGE')['default.payment.next.month'].count()
# test

copy_df = credit.copy()
l = copy_df.loc[lambda c : c['AGE'] >= 70]
default_filter = l.loc[lambda f : f['default.payment.next.month'].isin([0])]
percentage_seventy = (default_filter.shape[0] / l.shape[0]) * 100
print("The percentage of users don't have a default payment among all users at least 70: {:0.2f}%".format(percentage_seventy))

The percentage of users don't have a default payment among all users at least 70: 72.00%


Source: [Filter rows with operator chaining](https://)

##### Question 6: ( 2 points )
Among all users younger than 25 years,
What percentage of users don't have a default payment? 

In [9]:
## Question 6
## YOUR CODE HERE
# test1 = ((credit['AGE'] <= 25).sum() / credit['default.payment.next.month'].isin([0]).sum()) * 100
# test1

copy_df1 = credit.copy()
l1 = copy_df1.loc[lambda c : c['AGE'] < 25]
default_filter1 = l1.loc[lambda f : f['default.payment.next.month'].isin([0])]
percentage_twentyfive = (default_filter1.shape[0] / l1.shape[0]) * 100
print("The percentage of users don't have a default payment among all users younger than 25 : {:0.2f}%".format(percentage_twentyfive))

The percentage of users don't have a default payment among all users younger than 25 : 72.81%


Suppose you are given a model that predicts if a user will have a default payment in the next month or not. We want to analyze the privacy implications associated with these predictions.

We aren't particularly concerned about the specifics of the model. We want to use a simple model and analyze the privacy implications. 

In order to do this, we will create a model that predicts if a user will have a default payment in the next month or not using logistic regression. 

Additional resource: <https://machinelearningmastery.com/logistic-regression-tutorial-for-machine-learning/>

The model takes in input features (features 2-24). 
The 25th feature, 'default.payment.next.month' is used as the ground truth for the predictions. 
The output is the prediction (if a user will have a default payment in the next month). 

Some caveats about the model implementation:
- There can be better models used for this prediction task.
- Feature scaling can be done in a meaningful way.
- Feature engineering can be done to modify features based on domain knowledge.


The cell below takes care of the model creation, training, and testing for you. You don't have to write any code in this cell. 

In [10]:
## 'y' represents the ground truth for the predictions
y = credit['default.payment.next.month'].copy()

## Columns that represent the input features
features = ['LIMIT_BAL','SEX','EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

## 'X' represents the input data
X = credit[features].copy()

## Splitting the dataset into a training and testing datasets
## The model is trained using the training dataset
## The model is tested using the testing dataset 
## ( data that the model hasn't seen before)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Let's define 'credit_test' for further analyses
## credit_test is the subset of the dataframe used for testing
credit_test = pd.DataFrame(X_test,columns=features)

## Standardizing the features
scX = StandardScaler()
X_train = scX.fit_transform(X_train)
X_test = scX.transform(X_test)

## Defining the classifier
classifier = LogisticRegression(solver='lbfgs')

## Training the classifer
classifier.fit(X_train, y_train)

## Predictions using the testing data inputs
## 'y_pred' contains the predictions
y_pred = classifier.predict(X_test)

## Evaluating the accuracy of the model
cm = confusion_matrix( y_test, y_pred )
print("Accuracy on Test Set = %.2f" % ((cm[0,0] + cm[1,1] )/len(X_test)))

## Adding the predictions to the credit_test dataframe
## We will be using this dataframe 
credit_test['y_test']=y_test
credit_test['y_pred']=y_pred


Accuracy on Test Set = 0.81


Let's look at the 'credit_test' dataframe. Notice the 'y_test' column ( this represents the ground truth ) and 'y_pred' column ( this represents the model predictions )

In [11]:
credit_test

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,y_test,y_pred
2308,30000.0,1,2,2,25,0,0,0,0,0,0,8864.0,10062.0,11581.0,12580.0,13716.0,14828.0,1500.0,2000.0,1500.0,1500.0,1500.0,2000.0,0,0
22404,150000.0,2,1,2,26,0,0,0,0,0,0,136736.0,125651.0,116684.0,101581.0,77741.0,77264.0,4486.0,4235.0,3161.0,2647.0,2669.0,2669.0,0,0
23397,70000.0,2,3,1,32,0,0,0,0,0,0,70122.0,69080.0,68530.0,69753.0,70111.0,70212.0,2431.0,3112.0,3000.0,2438.0,2500.0,2554.0,0,0
25058,130000.0,1,3,2,49,0,0,0,0,0,-1,20678.0,18956.0,16172.0,16898.0,11236.0,6944.0,1610.0,1808.0,7014.0,27.0,7011.0,4408.0,0,0
2664,50000.0,2,2,2,36,0,0,0,0,0,2,94228.0,47635.0,42361.0,19574.0,20295.0,19439.0,2000.0,1500.0,1000.0,1800.0,0.0,1000.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,360000.0,2,1,2,27,1,-2,-1,-1,-1,-1,0.0,0.0,830.0,0.0,1271.0,179.0,0.0,830.0,0.0,1271.0,179.0,1970.0,0,0
14144,500000.0,2,1,2,28,2,0,0,2,0,0,98541.0,102052.0,111690.0,78070.0,78376.0,80912.0,6000.0,13151.0,0.0,3000.0,5000.0,10000.0,1,0
23108,60000.0,2,1,1,28,0,0,0,0,0,2,46233.0,47263.0,48696.0,50385.0,52045.0,52661.0,2100.0,2500.0,2500.0,2500.0,1600.0,1500.0,0,0
25703,20000.0,2,1,2,29,1,-1,-1,-1,-1,-1,0.0,2494.0,3967.0,1364.0,600.0,0.0,2494.0,3967.0,1370.0,600.0,0.0,0.0,0,0


##### Question 7:  ( 2 points )

How many data entry (rows) are present in the 'credit_test' dataframe

In [12]:
## Question 7
## YOUR CODE HERE
credit_test.shape[0]

6000

For this prediction model, no default payment can be considered as a positive outcome.

Therefore 'y_pred = 0' is a positive outcome.

To analyze if this model is fair, \
Let's look at one definition of mathematical fairness.

Demographic parity: 

Demographic Parity states that each segment of a protected class (e.g. gender, age, race) should receive the positive outcome at equal rates

ie the Positive Rate for group A should be the same as the Positive Rate for group B.




First, let's consider two groups: \
Group A: users with age >= 50 \
Group B: users with age < 50

##### Question 8:  ( 2 points )

In 'credit_test',
Among all users atleast 50 years old,
What percentage of users don't have a default payment prediction? 

In [13]:
credit_test.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'y_test',
       'y_pred'],
      dtype='object')

In [14]:
## Question 8:
## YOUR CODE HERE
copy_df_test = credit_test.copy()
new_frame = copy_df_test.loc[lambda c : c['AGE'] >= 50]
frame_filter = new_frame.loc[lambda f : f['y_pred'].isin([0])]
percentage_atleast_fifty = (frame_filter.shape[0] / new_frame.shape[0]) * 100
print("The percentage of users don't have a default payment among all users at least 50: {:0.2f}%".format(percentage_atleast_fifty))

The percentage of users don't have a default payment among all users at least 50: 88.62%


##### Question 9:  ( 2 points )

In 'credit_test',
Among all users younger than 50 years old,
What percentage of users don't have a default payment prediction? 

In [15]:
## Question 9:
## YOUR CODE HERE
copy_df_test = credit_test.copy()
new_frame = copy_df_test.loc[lambda c : c['AGE'] < 50]
frame_filter = new_frame.loc[lambda f : f['y_pred'].isin([0])]
percentage_younger_fifty = (frame_filter.shape[0] / new_frame.shape[0]) * 100
print("The percentage of users don't have a default payment among all users younger than 50: {:0.2f}%".format(percentage_younger_fifty))

The percentage of users don't have a default payment among all users younger than 50: 92.92%


We notice that the computed quantities are the Positive Rates for the respective groups. 

Now let's consider two groups, \
Group C: users with age >=70 
Group D: users with age < 25

##### Question 10:  ( 2 points )

In 'credit_test',
Among all users atleast 70 years old,
What percentage of users don't have a default payment prediction? 

In [16]:
## Question 10:
## YOUR CODE HERE
new_frame = copy_df_test.loc[lambda c : c['AGE'] >= 70]
frame_filter = new_frame.loc[lambda f : f['y_pred'].isin([0])]
percentage_atleast_seventy = (frame_filter.shape[0] / new_frame.shape[0]) * 100
print("The percentage of users don't have a default payment among all users at least 70: {:0.2f}%".format(percentage_atleast_seventy))

The percentage of users don't have a default payment among all users at least 70: 33.33%


##### Question 11:  ( 2 points )

In 'credit_test',
Among all users younger than 25 years old,
What percentage of users don't have a default payment prediction? 

In [17]:
## Question 11
## YOUR CODE HERE
new_frame = copy_df_test.loc[lambda c : c['AGE'] < 25]
frame_filter = new_frame.loc[lambda f : f['y_pred'].isin([0])]
percentage_younger_25 = (frame_filter.shape[0] / new_frame.shape[0]) * 100
print("The percentage of users don't have a default payment among all users younger than 25: {:0.2f}%".format(percentage_younger_25))

The percentage of users don't have a default payment among all users younger than 25: 89.23%


We can observe that the Positive Rate for group D is much higher than the Positive Rate for group C. Therefore, the model predictions don't satisfy demographic parity for groups C and D. 

From this idea of demographic parity, we can infer that the model's outputs are not similar for different demographic groups under consideration.

If fairness is achieved through demographic parity, the positive rates for each demographic group would be similar. The model's predictions can't be used to identify a demographic group definitely. 

If fairness is not acheived, depending on the group and the model, even if the input data is completely anonymized, the output from the model can be used to identify users. The predicted output can be used as a quasi-identifier in combination with external information. 

##### Question 12:  ( 3 points )

(a) Suggest 1 method through which privacy can be enhanced or preserved at the input data step. 

(b) Suggest 1 method through which privacy can be enhanced or preserved at the model training or model testing step. 

(c) Suggest 1 method through which privacy can be enhanced or preserved at the output/prediction step. 

Note: this is an open ended question and is an active area of research. Feel free to suggest techniques/methods that you think may work and justify very briefly. 

***

Enter your answer in the 'Submitted answers' cell.

***

( Double click the cell below to edit it )

***

# Submitted answers:


* Q.1 : [2] No. of rows - <b>30000</b>


* Q.2 : [2] No. of users atleast 70 years old - <b>25</b>


* Q.3 : [2] No. of users younger than 25 years - <b>2685</b>


* Q.4 : [2] % of users who didn't have a default payment -  <b>77.88%</b>


* Q.5 : [2] Among users atleast 70 years old, % who didn't have a default payment - <b>72.00%</b>


* Q.6 : [2] Among users younger than 25 years, % who didn't have a default payment - <b>72.81%</b>


* Q.7 : [2] No. of rows - <b>6000</b>


* Q.8 : [2]  Among users atleast 50 years old, % who didn't have a default payment  - <b>88.62%</b>


* Q.9 : [2] Among users younger than 50 years, % who didn't have a default payment - <b>92.92%</b>


* Q.10 : [2] Among users atleast 70 years old, % who didn't have a default payment -  <b>33.33%</b>


* Q.11 : [2] Among users younger than 25 years, % who didn't have a default payment - <b>89.23%</b>


* Q.12 : [3] <b></b>


 - (a) One method to enhance privacy technique at the input data step is through Crytographic algorithms – Homomorphic encryption: method that enables computational operations on encrypted data. It generates an encrypted result which, when decrypted, matches the result of the operations as if they had been performed on unencrypted data (i.e. plaintext). Source: [Privacy Enhancing Technique](https://research.aimultiple.com/privacy-enhancing-technologies/)
 
 
 - (b) One method can be perserved at model training is the coopetitive method. The method setting would provides a stronger security because the aggregator does not see the individual updates, nor do the parties see the intermediate aggregated models – all of these are exchanged in encrypted form and kept that way. Source: [Model Training](https://www.ericsson.com/en/blog/2020/2/training-a-machine-learning-model)
 
 
 - (c) One method can be perserved at output step is Secure Multi-Party Computation. The method that allows separate parties to jointly compute a common function while keeping both the inputs and the function parameters private. It allows a model to be trained or applied to data from different sources without disclosing the training data items or the model’s weights. Source: [Output step](https://blog.openmined.org/private-machine-learning-explained/)











